# PREPROCESAMIENTO DE DATOS
La idea de este notebook es mostrar cómo afecta la manera de redimensionar las imágenes al resultado de la clasificación. Se han evaluado tres maneras diferentes de redimensionar los datos: 

- Crop: Recortando una imagen cuadrada desde las esquinas utilizando la dimensión menor
- FillBoders: La dimensión menor es rellenada replicando bordes a ambos lados
- Resize: Se hace el redimensionamiento tal cual, deformando la imagen para hacerla cuadrada

## Importar todas las utilidades necesarias

In [19]:
from src.loadData import loadPolenData
from keras import backend as K
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras.callbacks import TensorBoard
import plotly
import plotly.graph_objs as go



## Cargar set de datos
A partir del tamaño deseado, el tipo de redimensionamiento y el número de imágenes de test que se obtendrán del dataset, se cargan el conjunto de entrenamiento y de test utilizando el formato Keras

In [20]:
def loadDataset(resizeType, imgSize, nTest, pathDataset, pathOut):
        return loadPolenData(pathDataset, pathOut + resizeType,
                             resizeType, imgSize, nTest)

## Definir el modelo
En esta función se define el modelo que se utilizará para realizar las pruebas

In [21]:
def defineModel(imgSize, nClass):
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, border_mode='same',
                            input_shape=(imgSize, imgSize,3)))
    model.add(Activation('relu'))
    
    model.add(Convolution2D(32, 3, 3))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(Activation('relu'))
    
    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nClass))
    model.add(Activation('softmax'))
    
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model

## Entrenar modelo
Esta función es la encargada de entrenar el modelo generado. 
Se utiliza aumento de datos a partir del conjunto de entrenamiento. Además, se utilizan tantos samples como tamaño tiene el conjunto de entrenamiento para cada iteración. El entrenamiento es llevado a cabo por lotes para evitar problemas de computación

In [22]:
def trainModel(model, xTrain, yTrain, xTest, yTest, nEpochs, logPath):
    tb = TensorBoard(
        log_dir=logPath, 
        histogram_freq=0, 
        write_graph=True, 
        write_images=False)
    
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,
        fill_mode ="nearest")  # randomly flip images

    datagen.fit(xTrain)
    hist = model.fit_generator(datagen.flow(xTrain, yTrain, batch_size=32), 
                        samples_per_epoch=xTrain.shape[0], 
                        #samples_per_epoch=750,
                        nb_epoch=nEpochs, 
                        verbose=2,
                        callbacks=[tb],
                        validation_data=(xTest, yTest))
    return hist, model

## Mostrar resultados
Esta función es la encargada de representar el la precisión de clasificación en el conjunto de test por cada iteración

In [23]:
def plotResults(resizeTypes, resultsAcc, resultsLoss):
    plotly.offline.init_notebook_mode(connected=True)
    fig = plotly.tools.make_subplots(rows=2, cols=1, subplot_titles=("Val_Acc", "Val_Loss"))
    for i in range(len(resizeTypes)):
        fig.append_trace(go.Scatter(y=resultsAcc[i], name=resizeTypes[i]), 1, 1)
        fig.append_trace(go.Scatter(y=resultsLoss[i], name=resizeTypes[i]), 2, 1)
    
    fig["layout"]["xaxis1"].update(title="Ciclo")
    fig["layout"]["xaxis2"].update(title="Ciclo")
    fig["layout"]["yaxis1"].update(title="Val_Acc")
    fig["layout"]["yaxis2"].update(title="Val_Loss")
    
    plotly.offline.iplot(fig)
    

## Ejecutar pruebas
Se redimensionan las imágenes a 100x100. Para el conjunto de test se toman 5 imágenes de cada clase. El número de iteraciones utilizado es 300, aunque como se podrá comprobar, el mejor modelo no es obtenido en esta iteración sino en anteriores. 

In [24]:
resizeTypes = ["CROP","FILLBORDERS",   "RESIZE"]
nClass = 23
imgSize = 100
nTest = 5
nEpochs = 300

logPath = "./logs"
pathDataset = "./dataset"
pathOut = "./dataset_"

resultsAcc = []
resultsLoss = []

for resizeType in resizeTypes:
    K.clear_session()
    print(resizeType)
    print("___________")
    (xTrain, yTrain), (xTest, yTest) = loadDataset(resizeType, imgSize, nTest, pathDataset, pathOut)
    model = defineModel(imgSize, nClass)
    hist, model = trainModel(model, xTrain, yTrain, xTest, yTest,nEpochs ,logPath)
    resultsAcc.append(hist.history["val_acc"])
    resultsLoss.append(hist.history["val_loss"])
    
    score, acc = model.evaluate(xTest, yTest, batch_size=32)
    print('Test score:', score)
    print('Test accuracy:', acc)
    
    model = None
    

CROP
___________
Epoch 1/300
5s - loss: 3.2543 - acc: 0.0594 - val_loss: 3.0256 - val_acc: 0.0696
Epoch 2/300
5s - loss: 2.9876 - acc: 0.0819 - val_loss: 2.8286 - val_acc: 0.0957
Epoch 3/300
5s - loss: 2.8383 - acc: 0.1249 - val_loss: 2.6817 - val_acc: 0.2000
Epoch 4/300
5s - loss: 2.6520 - acc: 0.1611 - val_loss: 2.4855 - val_acc: 0.2174
Epoch 5/300
5s - loss: 2.5037 - acc: 0.1986 - val_loss: 2.1157 - val_acc: 0.3391
Epoch 6/300
5s - loss: 2.3742 - acc: 0.2184 - val_loss: 2.1140 - val_acc: 0.3652
Epoch 7/300
5s - loss: 2.3156 - acc: 0.2471 - val_loss: 1.9738 - val_acc: 0.4174
Epoch 8/300
5s - loss: 2.2514 - acc: 0.2737 - val_loss: 1.8951 - val_acc: 0.4087
Epoch 9/300
5s - loss: 2.1757 - acc: 0.2942 - val_loss: 1.9590 - val_acc: 0.3478
Epoch 10/300
5s - loss: 2.0195 - acc: 0.3331 - val_loss: 1.6739 - val_acc: 0.4609
Epoch 11/300
5s - loss: 2.0065 - acc: 0.3447 - val_loss: 1.7974 - val_acc: 0.4087
Epoch 12/300
5s - loss: 1.9010 - acc: 0.3775 - val_loss: 1.6412 - val_acc: 0.4696
Epoch 13

## Resultados
Se puede observar como con esta configuración, el mejor resultado obtenido es el realizado utilizando el redimensionamiento "Crop", es decir, tomando una imagen cuadrada a partir de la dimensión menor. La precisión de clasificación en el conjunto de test en el mejor de los casos está en torno al 95%. 
 

In [25]:
plotResults(resizeTypes, resultsAcc, resultsLoss)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]

